### 데이터 load

In [ ]:
import pandas as pd
import nltk

In [ ]:
dataset = pd.read_excel('./Data/Covid_News_v3_0928.xlsx',encoding='utf-8', index_col=0)

In [ ]:
documents = dataset.Article.tolist()

In [ ]:
len(documents)

### 텍스트 전처리

In [ ]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [ ]:
news_df['clean_doc'][1]

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
stop_words.extend(['said','told','coronavirus','corona','covid','covid-19'])
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
tokenized_doc

from nltk.stem import PorterStemmer, LancasterStemmer

st1 = PorterStemmer()
st2 =  LancasterStemmer()

tokenized_doc = tokenized_doc.apply(lambda x: [st1.stem(w) for w in x])

tokenized_doc

In [ ]:
from nltk.stem import WordNetLemmatizer

lm = WordNetLemmatizer()

tokenized_doc = tokenized_doc.apply(lambda x: [lm.lemmatize(w) for w in x])

In [ ]:
tokenized_doc

In [ ]:
from tqdm import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
from nltk.tag import pos_tag

tokenized_document = tokenized_doc.progress_apply(lambda x: pos_tag(x))

import pickle as pkl

with open('./data/tokenized_document_5k.pkl','wb') as f:
    pkl.dump(document_filtered,f)

with open('./data/tokenized_document_5k.pkl','rb') as f:
    tokenized_doc = pkl.load(f)

In [ ]:
document_filtered = tokenized_document.apply(lambda x: [w[0] for w in x if len(w)==2 and (w[1] in ["NN","NNS","VB","VBG","VBD"])])

In [ ]:
tokenized_doc = document_filtered

In [ ]:
tokenized_df = pd.DataFrame(tokenized_doc)

In [ ]:
with open('./Data/tokenized_doc_NV','wb') as f:
    pkl.dump(tokenized_df,f)

### 토픽 모델링(LDA)

#### 정수 인코딩과 단어 집합 만들기

In [ ]:
tokenized_doc[:5]

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[2]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

In [ ]:
print(dictionary[5])

In [ ]:
len(dictionary)

#### 모델 훈련

In [ ]:
import gensim
num_topics = 15 #20개의 토픽, k=20

ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics = num_topics,
    id2word=dictionary,
    passes=20, 
    iterations=400,
    chunksize = 2000
)

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
import pickle as pkl

In [ ]:
with open('Result_v10/ldamodel.pkl','wb') as f:
    pkl.dump(ldamodel, f)

In [ ]:
with open('Result_v9/ldamodel.pkl','rb') as f:
    ldamodels = pkl.load(f)

In [ ]:
for topic in ldamodel.print_topics(num_words=5):
    print(topic)

In [ ]:
top_topics = ldamodel.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

### LDA 시각화

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary,sort_topics=False)
pyLDAvis.display(vis)

In [ ]:
pyLDAvis.save_html(vis, "./Result_v9/save_lda_topic.html")

### 문서 별 토픽 분포 보기

In [ ]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
import pandas as pd

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['No', 'top_topic', 'top_weight', 'weight']
topictable[:10]

In [ ]:
for i in range(15):
    globals()['variable{}'.format(i)] = topictable[topictable.top_topic==i].sort_values(by=['top_weight'],axis=0,ascending=False)
    globals()['variable{}'.format(i)].to_excel('./Result_v10/topic_result_' + str(i) + '.xlsx')

In [ ]:
topic_keyword = pd.DataFrame()

for i in range(15):
    topic_terms = ldamodel.show_topic(i,topn=60)
    keyword_list =[]
    for topic in topic_terms:
        keyword = topic[0]
        keyword_list.append(keyword)
    topic_keyword['topic_'+str(i+1)] = keyword_list

In [ ]:
topic_keyword

In [ ]:
topic_keyword.to_excel('./Result_v10/topic_keyword.xlsx')